In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import requests
import sys

from pathlib import Path

In [3]:
# Add root folder of the rag_bootcamp repo to PYTHONPATH
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.insert(0, str(parent_dir))

from utils.load_secrets import load_env_file
load_env_file()

In [4]:
GENERATOR_BASE_URL = os.environ.get("OPENAI_BASE_URL")
OPEN_API_KEY = os.environ.get("OPEN_API_KEY")

GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
EMBEDDING_MODEL_NAME = "BAAI/bge-base-en-v1.5"

In [5]:
# Load docs
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split (make chunks)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

model_kwargs = {'device': 'cuda', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

# Set embeddings model
print(f"Setting up the embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

# Store splits(=chunks)
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Setting up the embeddings model...


In [6]:
# LLM
llm = ChatOpenAI(model=GENERATOR_MODEL_NAME,  # 사용할 모델 이름
                 temperature=0,
                 base_url=GENERATOR_BASE_URL,
                 )

In [7]:
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 정의 (LangChain Hub의 템플릿을 로컬에서 구현): docs에는 hub 템플릿 사용
retrieval_qa_chat_prompt = PromptTemplate(
    input_variables=["context", "input"],
    template=(
        "Use the following context to answer the question.\n\n"
        "Context:\n{context}\n\n"
        "Question: {input}\n\n"
        "Answer:"
    ),
)

### LangChain Expression Language (LCEL) 형식으로 구현

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "input": RunnablePassthrough(),
    }
    | retrieval_qa_chat_prompt
    | llm
    | StrOutputParser()
)

response = qa_chain.invoke("What are autonomous agents?")

print(response)

Autonomous agents are systems that can perform tasks independently, making decisions and taking actions based on their programming and environment. They are equipped with the ability to plan ahead, decompose complex tasks into manageable steps, and reflect on their past actions to improve their performance over time. In the context of the provided text, autonomous agents are powered by Large Language Models (LLMs) and are capable of simulating human-like behavior in virtual environments, such as the Generative Agents experiment.


위 구현 방식은 **포괄적(verbose)**임.

이 composition logic을 
1. 커스터마이즈하고 helper function으로 감싸거나
2. 상위 수준의 `create_retrieval_chain`과 `create_stuff_documents_chain helper` 함수를 사용할 수 있다.

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(vectorstore.as_retriever(), combine_docs_chain)

response = rag_chain.invoke({"input": "What are autonomous agents?"})

# response는 json object형태로 반환되며, 그 중 'answer'가 LLM의 답변이다.
print(response['answer'])

Autonomous agents are systems that can perform tasks independently, making decisions and taking actions based on their programming and environment. They are equipped with the ability to plan ahead, decompose complex tasks into manageable steps, and reflect on their past actions to improve their performance over time. In the context of the provided text, autonomous agents are powered by Large Language Models (LLMs) and are capable of simulating human-like behavior in virtual environments, such as the Generative Agents experiment.
